# PlayHQ Fixture Scraping

This system allows to scrape games from [PlayHQ](http://playhq.com/
) via its Public [API](https://support.playhq.com/hc/en-au/sections/4405422358297-PlayHQ-APIs) and produce CSV files ready to be uploaded to [TeamApp](https://brunswickmagicbasketball.teamapp.com/). (Private APIs are not available to clubs and associations.)

The *Public* APIs only require a header parameters to get a successful response, which includes `x-api-key` (also referred to as the Client ID) and `x-phq-tenant` (refers to the sport/association - in this case `bv`).

Detailed reference documentation for PlayHQ API can be found [here](https://docs.playhq.com/tech).

Contact Sebastian Sardina  (sssardina@gmail.com)

In [1]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import json
import pandas as pd
import re
import csv
import os
import calendar, datetime

import playhq_api as phq

# import dtale

## 1. Set-up application

We start by creating a connection to the PlayHQ Public API.

To do so, we need to define the ids and keys for the club to access the PlayHQ API.

The organization Play-HQ ID can be obtained from the admin PlayHQ URL:

```
https://bv.playhq.com/org/<ORG_ID>/competitions
```

The *Public* APIs only require the below header parameters to get a successful response:

- `x-api-key` (also referred to as the Client ID) will be provided by PlayHQ when you request access to the public API via their [support page](https://support.playhq.com/hc/en-au) or email support@playhqsupport.zendesk.com.
- `x-phq-tenant` usually refers to the sport/association - in this case '`bv`'.

The feature to create new API credentials is disabled for you and can only be actioned by a Super Administrator role within the Play HQ portal. Please use the credentials provided to call the PlayHQ public APIs.

In [14]:
CLUB_NAME = "Brunswick Magic Basketball Club"
X_TENANT="bv"
ORG_ID="8c4d5431-eaa5-4644-82ac-992abe224b88"
GAME_DATE = phq.next_day(calendar.SATURDAY)

# Provide the x-api-key either explicitly or via a file
X_API_KEY="<YOUR PLAYHQ ADMIN KEY>"

if os.path.exists('x_api_key.txt'):
    with open('x_api_key.txt') as f:
        X_API_KEY = f.readlines()[0]

phq_club = phq.PlayHQ(CLUB_NAME, ORG_ID, X_API_KEY, X_TENANT)

print(f"Set-up games for club {CLUB_NAME} for upcoming Saturday is: {GAME_DATE.strftime("%A %B %d of %Y")}")


Set-up games for club Brunswick Magic Basketball Club for upcoming Saturday is: 2022-08-06


'Saturday August 06 of 2022'

Next define the data for the season that we are interested in.

In [19]:
SEASON = "Winter 2022"

DESC_BYE_TAPP = "Sorry, no game for the team in this round."
DESC_TAPP = """RSVP is YES by default - if you cannot make it, please let your Team Manager know as soon as possible.
Opponent: {opponent}
Venue: {venue} ({court})
Address: {address} {address_tips}
Google Maps coord: https://maps.google.com/?q={coord}

- Please ensure you arrive early and ready.
- Remember that shorts should have no pockets, players should not wear bracelets/watch as it is a risk of injury.
- 45 min schedule with 18 min halves.
- Each team needs to provide a scorer. TMs, please consider a roster.
- Players should not bring balls into the venue - game balls provided by Magic in coach's equipment bag.
- Beginners refs will be wearing green shirts. Please support and respect them through a POSITIVE sideline behaviour.

Check the game in PlayHQ: {url_game}
Check the round in PlayHQ: {url_grade}
All clubs in PlayHQ: https://bit.ly/bmbc-w22
"""

PATH_SAVE_CSV='Brunswick_Magics/2022.01.Winter/fixture'

## 2. Get the teams and upcoming games for those teams

First, get the competition id for the season the organisation is in.

In [20]:
season_id = phq_club.get_season_id(SEASON)
teams_df = phq_club.get_season_teams(season_id)
teams_df

,id,name,grade.id,grade.name,grade.url,age
0,d7486008-b2fe-47db-8011-01265eaf1cfe,Magic U8 Mixed Purple,be7076cb-a1de-4033-8082-b52c8c149861,Saturday U8 Mixed Division 1/2,https://www.playhq.com/basketball-victoria/org...,8
1,8d635867-0718-4f70-a1b8-1a224992d294,Magic U16 Girls Gold,19b86372-2ad5-4a5a-9592-467f826f7d63,Saturday U16 Girls Division 1/2,https://www.playhq.com/basketball-victoria/org...,16
2,69d69214-641d-4ece-a2ce-ca9534865553,Magic U16 Boys Purple,be4b2b9c-0bc1-44e2-ac3a-b89bb10230f8,Saturday U16 Boys Division 3,https://www.playhq.com/basketball-victoria/org...,16
3,135c3bab-3004-4426-b9c6-3669b48ce271,Magic U16 Boys Gold,ceac1ec4-c86b-47c6-9e67-124d923ea713,Saturday U16 Boys Division 2,https://www.playhq.com/basketball-victoria/org...,16
4,99f369ed-5c63-40e7-8591-8cd18fba7ec0,Magic U14 Boys Gold,d900616b-0f52-4d3e-8090-4eaf877491bb,Saturday U14 Boys Division 5,https://www.playhq.com/basketball-victoria/org...,14
5,8b2bc68b-9fda-435c-9c13-297d1d3187d1,Magic U14 Girls Purple,b0785b1f-bec6-448b-a76d-83e1a6371fb3,Saturday U14 Girls Division 3,https://www.playhq.com/basketball-victoria/org...,14
6,9ebeb141-4023-4744-84f1-f58957b966b2,Magic U14 Girls Gold,b0785b1f-bec6-448b-a76d-83e1a6371fb3,Saturday U14 Girls Division 3,https://www.playhq.com/basketball-victoria/org...,14
7,2b844e90-6575-47bc-bb49-f55840f8e6f8,Magic U14 Girls Black,237a9d9a-52c5-4136-b179-c01b10ad93ae,Saturday U14 Girls Division 4,https://www.playhq.com/basketball-victoria/org...,14
8,623c00bb-302b-4492-a988-90aacc3e7768,Magic U14 Boys Purple,d900616b-0f52-4d3e-8090-4eaf877491bb,Saturday U14 Boys Division 5,https://www.playhq.com/basketball-victoria/org...,14
9,a1e571f6-8ac4-49d2-9787-f62b35947eca,Magic U14 Boys Black,b877a73b-037d-460b-acb5-a2c45e5debee,Saturday U14 Boys Division 2,https://www.playhq.com/basketball-victoria/org...,14


Next get all upcoming games for the Club's teams.

In [21]:
upcoming_games_df = phq_club.get_games_by_status(teams_df)
upcoming_games_df[phq.GAMES_COLS]

2022-08-03 22:15:44 INFO Games extracted for team: Magic U8 Mixed Purple
2022-08-03 22:15:44 INFO Games extracted for team: Magic U16 Girls Gold
2022-08-03 22:15:44 INFO Games extracted for team: Magic U16 Boys Purple
2022-08-03 22:15:45 INFO Games extracted for team: Magic U16 Boys Gold
2022-08-03 22:15:45 INFO Games extracted for team: Magic U14 Boys Gold
2022-08-03 22:15:45 INFO Games extracted for team: Magic U14 Girls Purple
2022-08-03 22:15:46 INFO Games extracted for team: Magic U14 Girls Gold
2022-08-03 22:15:46 INFO Games extracted for team: Magic U14 Girls Black
2022-08-03 22:15:46 INFO Games extracted for team: Magic U14 Boys Purple
2022-08-03 22:15:47 INFO Games extracted for team: Magic U14 Boys Black
2022-08-03 22:15:47 INFO Games extracted for team: Magic U12 Girls Gold
2022-08-03 22:15:48 INFO Games extracted for team: Magic U12 Girls Black
2022-08-03 22:15:48 INFO Games extracted for team: Magic U12 Boys Purple
2022-08-03 22:15:48 INFO Games extracted for team: Magic U

,team_name,status,schedule.date,schedule.time,venue.name
0,U8 Mixed Purple,UPCOMING,2022-08-06,08:30:00,Coburg Basketball Stadium
1,U16 Girls Gold,UPCOMING,2022-08-06,13:00:00,Coburg Basketball Stadium
2,U16 Boys Purple,UPCOMING,2022-08-06,13:45:00,Dallas Brooks Community Primary School
3,U16 Boys Gold,UPCOMING,2022-08-06,13:00:00,Dallas Brooks Community Primary School
4,U14 Boys Gold,UPCOMING,2022-08-06,10:45:00,Northcote High School
5,U14 Girls Purple,UPCOMING,2022-08-06,13:00:00,Dallas Brooks Community Primary School
6,U14 Girls Gold,UPCOMING,2022-08-06,13:00:00,Dallas Brooks Community Primary School
7,U14 Girls Black,UPCOMING,2022-08-06,11:30:00,Dallas Brooks Community Primary School
8,U14 Boys Purple,UPCOMING,2022-08-06,11:30:00,Pascoe Vale Girls College
9,U14 Boys Black,UPCOMING,2022-08-06,12:15:00,Coburg Basketball Stadium


In [22]:
game_rounds = upcoming_games_df['round.name'].unique()
game_date = upcoming_games_df['schedule.date'].unique()
if len(game_date) == 1:
    game_date = pd.to_datetime(game_date[0])
else:
    print("There are more than one date!: ", game_date)
    exit(1)

print(f'Games extracted for rounds {game_rounds} on date: {game_date.strftime("%a %Y/%m/%d")}')

Games extracted for rounds ['Round 13' 'Round 7' 'Round 5' 'Round 9' 'Round 8'] on date: Sat 2022/08/06


## 3. Convert to Teams App CSV format

Next, we conver tthe PlayHQ upcoming games to Teams App format so we can produce a CSV file to be imported into Teams App.

In [23]:
games_tapps_df = phq.to_teamsapp_schedule(upcoming_games_df, desc_template=DESC_TAPP)
print("Done computing the games for Teams App")
games_tapps_df

Done computing the games for Teams App


,team_name,team_id,round.name,round.abbreviatedName,event_name,opponent,start_date,end_date,start_time,end_time,...,attendance_tracking,duty_roster,ticketing,reference_id,venue,court,geo,game_url,grade_url,description
0,U8 Mixed Purple,d7486008-b2fe-47db-8011-01265eaf1cfe,Round 13,R13,U8 Mixed Purple - Round 13,STARS U8 Mixed CH,2022-08-06,2022-08-06,08:30:00,09:15:00,...,0,1,0,,Coburg Basketball Stadium,Court 1,"(-37.73315,144.97684)",https://tinyurl.com/28fqqxd6,https://tinyurl.com/26un6t5r,RSVP is YES by default - if you cannot make it...
1,U16 Girls Gold,8d635867-0718-4f70-a1b8-1a224992d294,Round 13,R13,U16 Girls Gold - Round 13,Newlands U16 Girls LYNX,2022-08-06,2022-08-06,13:00:00,13:45:00,...,0,1,0,,Coburg Basketball Stadium,Court 4,"(-37.73315,144.97684)",https://tinyurl.com/22y5tyyn,https://tinyurl.com/25vwaujv,RSVP is YES by default - if you cannot make it...
2,U16 Boys Purple,69d69214-641d-4ece-a2ce-ca9534865553,Round 13,R13,U16 Boys Purple - Round 13,St Fidelis U16 Boys White,2022-08-06,2022-08-06,13:45:00,14:30:00,...,0,1,0,,Dallas Brooks Community Primary School,Court 2,"(-37.67041,144.9418)",https://tinyurl.com/25t2csb7,https://tinyurl.com/28weea2e,RSVP is YES by default - if you cannot make it...
3,U16 Boys Gold,135c3bab-3004-4426-b9c6-3669b48ce271,Round 13,R13,U16 Boys Gold - Round 13,Warriors U16 Boys Black,2022-08-06,2022-08-06,13:00:00,13:45:00,...,0,1,0,,Dallas Brooks Community Primary School,Court 1,"(-37.67041,144.9418)",https://tinyurl.com/2dxs7pot,https://tinyurl.com/2chydnza,RSVP is YES by default - if you cannot make it...
4,U14 Boys Gold,99f369ed-5c63-40e7-8591-8cd18fba7ec0,Round 13,R13,U14 Boys Gold - Round 13,Thunder U14 Boys Blue,2022-08-06,2022-08-06,10:45:00,11:30:00,...,0,1,0,,Northcote High School,Court 1,"(-37.7742,144.98985)",https://tinyurl.com/29tvvjmh,https://tinyurl.com/24br3e8r,RSVP is YES by default - if you cannot make it...
5,U14 Girls Purple,8b2bc68b-9fda-435c-9c13-297d1d3187d1,Round 13,R13,U14 Girls Purple - Round 13,Magic U14 Girls Gold,2022-08-06,2022-08-06,13:00:00,13:45:00,...,0,1,0,,Dallas Brooks Community Primary School,Court 2,"(-37.67041,144.9418)",https://tinyurl.com/26fq4wqh,https://tinyurl.com/2cbtlglg,RSVP is YES by default - if you cannot make it...
6,U14 Girls Gold,9ebeb141-4023-4744-84f1-f58957b966b2,Round 13,R13,U14 Girls Gold - Round 13,Magic U14 Girls Purple,2022-08-06,2022-08-06,13:00:00,13:45:00,...,0,1,0,,Dallas Brooks Community Primary School,Court 2,"(-37.67041,144.9418)",https://tinyurl.com/26fq4wqh,https://tinyurl.com/2cbtlglg,RSVP is YES by default - if you cannot make it...
7,U14 Girls Black,2b844e90-6575-47bc-bb49-f55840f8e6f8,Round 13,R13,U14 Girls Black - Round 13,STARS U14 Girls LK,2022-08-06,2022-08-06,11:30:00,12:15:00,...,0,1,0,,Dallas Brooks Community Primary School,Court 2,"(-37.67041,144.9418)",https://tinyurl.com/27gq7ddc,https://tinyurl.com/2bajtekn,RSVP is YES by default - if you cannot make it...
8,U14 Boys Purple,623c00bb-302b-4492-a988-90aacc3e7768,Round 13,R13,U14 Boys Purple - Round 13,Thunder U14 Boys Grey,2022-08-06,2022-08-06,11:30:00,12:15:00,...,0,1,0,,Pascoe Vale Girls College,Court 1,"(-37.71901,144.93687)",https://tinyurl.com/29b3g5c6,https://tinyurl.com/24br3e8r,RSVP is YES by default - if you cannot make it...
9,U14 Boys Black,a1e571f6-8ac4-49d2-9787-f62b35947eca,Round 7,R7,U14 Boys Black - Round 7,Piranhas U14 Boys Silver,2022-08-06,2022-08-06,12:15:00,13:00:00,...,0,1,0,,Coburg Basketball Stadium,Court 3,"(-37.73315,144.97684)",https://tinyurl.com/26776w9r,https://tinyurl.com/222c8cj7,RSVP is YES by default - if you cannot make it...


## 4. Prepare dataframe for Teams App CSV

First filter and re-order columns relevant to Teams App schedule format, and then produce corresponding CSV.

In [24]:
TAPP_COLS_CSV = ['event_name', 'team_name', 'start_date', 'end_date', 'start_time', 'end_time', 'description', 'venue', 'location', 'access_groups', 'rsvp', 'comments', 'attendance_tracking', 'duty_roster', 'ticketing']

team_apps_csv_df = games_tapps_df.loc[:, TAPP_COLS_CSV]
team_apps_csv_df.sample(4)


,event_name,team_name,start_date,end_date,start_time,end_time,description,venue,location,access_groups,rsvp,comments,attendance_tracking,duty_roster,ticketing
3,U16 Boys Gold - Round 13,U16 Boys Gold,2022-08-06,2022-08-06,13:00:00,13:45:00,RSVP is YES by default - if you cannot make it...,Dallas Brooks Community Primary School,"26-36 King Street, Dallas",U16 Boys Gold,1,1,0,1,0
11,U12 Girls Black - Round 5,U12 Girls Black,2022-08-06,2022-08-06,10:45:00,11:30:00,RSVP is YES by default - if you cannot make it...,Coburg Basketball Stadium,"25 Outlook Road, Coburg North",U12 Girls Black,1,1,0,1,0
4,U14 Boys Gold - Round 13,U14 Boys Gold,2022-08-06,2022-08-06,10:45:00,11:30:00,RSVP is YES by default - if you cannot make it...,Northcote High School,"19-25 St Georges Road, Northcote",U14 Boys Gold,1,1,0,1,0
10,U12 Girls Gold - Round 13,U12 Girls Gold,2022-08-06,2022-08-06,10:45:00,11:30:00,RSVP is YES by default - if you cannot make it...,Dallas Brooks Community Primary School,"26-36 King Street, Dallas",U12 Girls Gold,1,1,0,1,0


In [25]:
# Inspect description of one record
print(team_apps_csv_df.iloc[4]['event_name'])
print(team_apps_csv_df.iloc[4]['description'])


U14 Boys Gold - Round 13
RSVP is YES by default - if you cannot make it, please let your Team Manager know as soon as possible.
Opponent: Thunder U14 Boys Blue
Venue: Northcote High School (Court 1)
Address: 19-25 St Georges Road, Northcote 
Google Maps coord: https://maps.google.com/?q=(-37.7742,144.98985)

- Please ensure you arrive early and ready.
- Remember that shorts should have no pockets, players should not wear bracelets/watch as it is a risk of injury.
- 45 min schedule with 18 min halves.
- Each team needs to provide a scorer. TMs, please consider a roster.
- Players should not bring balls into the venue - game balls provided by Magic in coach's equipment bag.
- Beginners refs will be wearing green shirts. Please support and respect them through a POSITIVE sideline behaviour.

Check the game in PlayHQ: https://tinyurl.com/29tvvjmh
Check the round in PlayHQ: https://tinyurl.com/24br3e8r
All clubs in PlayHQ: https://bit.ly/bmbc-w22



### Extract and add BYE games

In [30]:
# Extract the date of the round
date = team_apps_csv_df.iloc[1]['start_date']

print(f"Extract BYE games for games on {date.strftime('%a %d/%m/%Y')}")

# Extract teams that do not have a game
playing_teams = games_tapps_df['team_id'].tolist()
bye_teams_df = teams_df.loc[~teams_df['id'].isin(playing_teams)].copy()

# Add entries for BYE teams
if not bye_teams_df.empty:  # there are BYE games
    print("There are BYE games...")

    bye_teams_df = bye_teams_df[['id', 'name']]

    bye_teams_df['team_name'] = bye_teams_df.apply(lambda x: re.search("U.*", x['name']).group(0), axis=1)
    bye_teams_df['access_groups'] = bye_teams_df['team_name']
    bye_teams_df['event_name'] = bye_teams_df['team_name'] + " - BYE"
    bye_teams_df['start_date'] = date
    bye_teams_df['end_date'] = date
    bye_teams_df['start_time'] = datetime.time(hour=0,minute=0,second=0)
    bye_teams_df['end_time'] = datetime.time(hour=0,minute=0,second=0)
    bye_teams_df['description'] = DESC_BYE_TAPP
    bye_teams_df['location'] = ""
    bye_teams_df['venue'] = "BYE"

    bye_teams_df['rsvp'] = 0
    bye_teams_df['comments'] = 0
    bye_teams_df['attendance_tracking'] = 0
    bye_teams_df['duty_roster'] = 0
    bye_teams_df['ticketing'] = 0
    bye_teams_df['reference_id'] = ""

    bye_teams_df = bye_teams_df[TAPP_COLS_CSV]
    team_apps_csv_df = pd.concat([team_apps_csv_df, bye_teams_df])
    team_apps_csv_df.drop_duplicates(inplace=True)

bye_teams_df

Extract BYE games for games on Sat 06/08/2022


0

Let's report the teams that have a BYE

In [33]:
if bye_teams_df.size > 0:
    for x in bye_teams_df['team_name'].values:
        print(x)
else:
    print("No BYE games this round...")

No BYE games this round...


In [34]:
team_apps_csv_df.columns
team_apps_csv_df[['team_name', 'start_time', 'venue']]

,team_name,start_time,venue
0,U8 Mixed Purple,08:30:00,Coburg Basketball Stadium
1,U16 Girls Gold,13:00:00,Coburg Basketball Stadium
2,U16 Boys Purple,13:45:00,Dallas Brooks Community Primary School
3,U16 Boys Gold,13:00:00,Dallas Brooks Community Primary School
4,U14 Boys Gold,10:45:00,Northcote High School
5,U14 Girls Purple,13:00:00,Dallas Brooks Community Primary School
6,U14 Girls Gold,13:00:00,Dallas Brooks Community Primary School
7,U14 Girls Black,11:30:00,Dallas Brooks Community Primary School
8,U14 Boys Purple,11:30:00,Pascoe Vale Girls College
9,U14 Boys Black,12:15:00,Coburg Basketball Stadium


## 5. Save to CSV file for Teams App import

Finally, we save the data to a CSV file that can be imported into the [SCHEDULE of TeamsApp for all Entries](https://brunswickmagicbasketball.teamapp.com/clubs/263995/events?_list=v1&team_id=all).

In [35]:
# date_games = team_apps_csv_df.iloc[1]['start_date']

file_csv = os.path.join(PATH_SAVE_CSV, f"schedule-teamsapp-{date.strftime('%Y_%m_%d')}.csv")

print(f'Saving schedule for round {game_rounds} on date: {game_date.strftime("%a %Y/%m/%d")}')
print('File to save TeamApp schedule:', file_csv)
team_apps_csv_df.to_csv(file_csv, index=False)

Saving schedule for round ['Round 13' 'Round 7' 'Round 5' 'Round 9' 'Round 8'] on date: Sat 2022/08/06
File to save TeamApp schedule: Brunswick_Magics/2022.01.Winter/fixture/schedule-teamsapp-2022_08_06.csv


# Get one team game

In [45]:
team = "Magic U14 Girls Gold"

team_id = teams_df.query('name == @team')['id'].values[0]


game_team_df = phq_club.get_team_fixture_df(team_id)
game_team_df.columns
game_team_df[['status', 'createdAt', 'updatedAt', 'schedule.date', 'schedule.time', 'venue.name']]

pd.to_datetime(game_team_df.iloc[11]['schedule.date'])

# upcoming_games_df = phq_club.get_games_by_status(teams_df)
# upcoming_games_df[]

Timestamp('2022-08-06 00:00:00')

# Testing area

In [ ]:
# short_url = shorten_url.short("https://www.playhq.com/basketball-victoria/org/coburg-basketball-association/junior-domestic-winter-2022/saturday-u-10-girls-division-2/game-centre/b8a62f18")
# short_url

# s = pyshorteners.Shortener()
# #  s.bitly.short('http://www.google.com')
# s.dagd.short('http://www.google.com')

# shorten_url('http://www.google.com')

In [ ]:
# id_team = teams_df.loc[teams_df['name'] == "Magic U12 Boys Black"]['id'].values[0]
# id_team

# data_json = phq_club.get_json(f'teams/{id_team}/fixture')
# games_team_df = pd.json_normalize(data_json['data'])

# games_team_df.columns
# upcoming_games = games_team_df.loc[games_team_df['status'] == "UPCOMING"]

# upcoming_games.empty
